## Imports ##

In [1]:
import pandas as pd
import torch
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset

In [2]:
#Loading Dataset
DATASET_PATH = "../Datasets/Covid_Cleaned.csv"
COVID_data = pd.read_csv(DATASET_PATH)

In [3]:
DATASET_PATH = "../Datasets/Covid_Cleaned.csv"
COVID_data = pd.read_csv(DATASET_PATH)
corr = COVID_data.corr(numeric_only=True)

threshold = 0.00

col_to_drop = corr['CLASIFFICATION_FINAL'][abs(corr['CLASIFFICATION_FINAL']) < threshold].index

# COVID_data.drop(col_to_drop, axis=1, inplace=True)

# COVID_data.describe()

## Batch preperation ##

In [4]:
X = COVID_data.drop(columns=['CLASIFFICATION_FINAL'])  # features (20)
y = COVID_data['CLASIFFICATION_FINAL']                 # target (low / high risk of COVID-19)

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.1, random_state=32) # 70% train, 30% test
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=32) # test -> 50% validation, 50% final test

In [5]:
# Creating Pytorch Tensors
X_train_tensor = torch.tensor(X_train.to_numpy(), dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.to_numpy(), dtype=torch.float32)

X_val_tensor = torch.tensor(X_val.to_numpy(), dtype=torch.float32)
y_val_tensor = torch.tensor(y_val.to_numpy(), dtype=torch.float32)

X_test_tensor = torch.tensor(X_test.to_numpy(), dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.to_numpy(), dtype=torch.float32)

In [6]:
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

In [7]:
batch_size = 1024

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## Model Training ##

In [8]:
import torch.nn as nn
from src.utils import OurModel


In [9]:
ml = OurModel()

In [10]:
# model configuration
config = nn.Sequential(
    nn.Linear(COVID_data.shape[1]-1, 8),
    nn.ReLU(),
    nn.Linear(8, 32),
    nn.Tanh(),
    nn.Linear(32, 16),
    nn.ReLU(),
    nn.Linear(16, 1),
)
ml.add_configuration(config)

In [11]:
#GPU optimalization TODO: Doesnt work
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
ml.to(device)

cuda


OurModel(
  (net): Sequential(
    (0): Linear(in_features=20, out_features=8, bias=True)
    (1): ReLU()
    (2): Linear(in_features=8, out_features=32, bias=True)
    (3): Tanh()
    (4): Linear(in_features=32, out_features=16, bias=True)
    (5): ReLU()
    (6): Linear(in_features=16, out_features=1, bias=True)
  )
)

In [12]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.RMSprop(ml.parameters(), lr=0.001)

In [13]:
# # Model Training
criterion = nn.BCEWithLogitsLoss()

epochs = 20
for epoch in range(epochs):
    ml.train()  # set model to training mode
    total_loss = 0
    correct = 0
    total = 0

    for inputs, true in train_loader:
        inputs, true = inputs.to(device), true.to(device)
        true = true.unsqueeze(1)
        y_pred = ml(inputs)
        loss = criterion(y_pred, true)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        preds = torch.sigmoid(y_pred)              # convert logits → probabilities
        predicted = (preds > 0.5).float()          # threshold to 0/1
        correct += (predicted == true).sum().item()
        total += true.size(0)
    # Epoch summary
    avg_loss = total_loss / len(train_loader)
    accuracy = correct / total

    print(f"Epoch [{epoch+1}/{epochs}] | Loss: {avg_loss:.4f} | Accuracy: {accuracy:.4f}")


Epoch [1/20] | Loss: 0.6668 | Accuracy: 0.5940
Epoch [2/20] | Loss: 0.6480 | Accuracy: 0.6209
Epoch [3/20] | Loss: 0.6432 | Accuracy: 0.6254
Epoch [4/20] | Loss: 0.6413 | Accuracy: 0.6275
Epoch [5/20] | Loss: 0.6409 | Accuracy: 0.6279
Epoch [6/20] | Loss: 0.6401 | Accuracy: 0.6274
Epoch [7/20] | Loss: 0.6393 | Accuracy: 0.6286
Epoch [8/20] | Loss: 0.6395 | Accuracy: 0.6287
Epoch [9/20] | Loss: 0.6383 | Accuracy: 0.6299
Epoch [10/20] | Loss: 0.6389 | Accuracy: 0.6293
Epoch [11/20] | Loss: 0.6382 | Accuracy: 0.6300
Epoch [12/20] | Loss: 0.6383 | Accuracy: 0.6296
Epoch [13/20] | Loss: 0.6380 | Accuracy: 0.6301
Epoch [14/20] | Loss: 0.6378 | Accuracy: 0.6307
Epoch [15/20] | Loss: 0.6376 | Accuracy: 0.6312
Epoch [16/20] | Loss: 0.6375 | Accuracy: 0.6308
Epoch [17/20] | Loss: 0.6368 | Accuracy: 0.6324
Epoch [18/20] | Loss: 0.6369 | Accuracy: 0.6322
Epoch [19/20] | Loss: 0.6369 | Accuracy: 0.6319
Epoch [20/20] | Loss: 0.6365 | Accuracy: 0.6328


## Sweep ##

In [14]:
import wandb
wandb.login()

wandb: Currently logged in as: xbukovinam (xbukovinam-faculty-of-informatics-and-infromation-techno) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [18]:
sweep_config = {
    "method": "random",  # or "grid", "bayes"
    "metric": {"name": "loss", "goal": "minimize"},
    "parameters": {
        "learning_rate": {"min": 0.0001, "max": 0.01},
        # "epochs": {"min": 2, "max": 20},
        "hidden_size_1": {"values": [32, 64, 128]},
        "hidden_size_2": {"values": [8, 32, 64]},
        "hidden_size_3": {"values": [8, 32, 64]},
        "hidden_size_4": {"values": [8, 32, 64]},
        "n_layers": {"values": [2, 4]},
        # "dropout": {"min": 0.0, "max": 0.5},
        "optimizer": {"values": ["Adam", "SGD"]},
        "activation": {"values": ["relu", "tanh"]},
        "batch_size": {"values": [1024, 2048, 4096]},
    }
}

In [19]:
def train():
    wandb.init()
    CONFIG = wandb.config
    train_loader = DataLoader(train_dataset, batch_size=CONFIG.batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=CONFIG.batch_size, shuffle=False)
    # test_loader = DataLoader(test_dataset, batch_size=CONFIG.batch_size, shuffle=False)


    hl_sizes = [
        CONFIG.hidden_size_1, CONFIG.hidden_size_2, CONFIG.hidden_size_3,
        CONFIG.hidden_size_4,
        # CONFIG.hidden_size_5, CONFIG.hidden_size_6,
        # CONFIG.hidden_size_7, CONFIG.hidden_size_8
                ]
    #Building the neural network from CONFIG
    sw_nn = OurModel()
    sw_nn.sweeping_build(COVID_data.shape[1]-1, 1, n_layers=CONFIG.n_layers, hidden_size=hl_sizes, activation_f=CONFIG.activation, dropout=None)
    optimizer = getattr(torch.optim, CONFIG.optimizer)(sw_nn.parameters(), lr=CONFIG.learning_rate)
    criterion = nn.BCEWithLogitsLoss()

    #GPU computing
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    sw_nn.to(device)

    for epoch in range(10):
        sw_nn.train()
        total_loss = 0
        correct = 0
        total = 0
        for inputs, true in train_loader:
            inputs, true = inputs.to(device), true.to(device)
            true = true.unsqueeze(1)
            optimizer.zero_grad()
            y_pred = sw_nn(inputs)
            loss = criterion(y_pred, true)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            preds = torch.sigmoid(y_pred)              # convert logits → probabilities
            predicted = (preds > 0.5).float()          # threshold to 0/1
            correct += (predicted == true).sum().item()
            total += true.size(0)

        avg_train_loss = total_loss / len(train_loader)
        accuracy = correct / total

        sw_nn.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, true in val_loader:
                inputs, true = inputs.to(device), true.to(device)
                preds = sw_nn(inputs)
                true = true.unsqueeze(1)
                loss = criterion(preds, true)
                val_loss += loss.item()
                predicted = (preds > 0.5).float()
                correct += (predicted == true).sum().item()
                total += true.size(0)
        avg_val_loss = val_loss / len(val_loader)
        accuracy = correct / total

        wandb.log({
            "epoch": epoch + 1,
            "train_loss": avg_train_loss,
            "val_loss": avg_val_loss,
            "val_accuracy": accuracy
        })


In [20]:
sweep_id = wandb.sweep(sweep=sweep_config, project="xbukovinam-faculty-of-informatics-and-infromation-techno")
wandb.agent(sweep_id, function=train, count=10)

Create sweep with ID: 4c83xer9
Sweep URL: https://wandb.ai/xbukovinam-faculty-of-informatics-and-infromation-techno/xbukovinam-faculty-of-informatics-and-infromation-techno/sweeps/4c83xer9


wandb: Agent Starting Run: 3hrnxamh with config:
wandb: 	activation: tanh
wandb: 	batch_size: 4096
wandb: 	hidden_size_1: 32
wandb: 	hidden_size_2: 32
wandb: 	hidden_size_3: 64
wandb: 	hidden_size_4: 8
wandb: 	learning_rate: 0.0004242559345708381
wandb: 	n_layers: 4
wandb: 	optimizer: Adam


epoch,▁▂▃▃▄▅▆▆▇█
train_loss,█▇▆▃▂▂▁▁▁▁
val_accuracy,▁▃▄▄▆███▇▇
val_loss,█▇▅▃▂▂▁▁▁▁
epoch,10
train_loss,0.6395
val_accuracy,0.57844
val_loss,0.63678


wandb: Agent Starting Run: wg1ps67i with config:
wandb: 	activation: relu
wandb: 	batch_size: 4096
wandb: 	hidden_size_1: 64
wandb: 	hidden_size_2: 8
wandb: 	hidden_size_3: 64
wandb: 	hidden_size_4: 8
wandb: 	learning_rate: 0.0029895198236937265
wandb: 	n_layers: 4
wandb: 	optimizer: Adam


epoch,▁▂▃▃▄▅▆▆▇█
train_loss,█▆▃▃▂▂▁▁▁▁
val_accuracy,▃▁▄▂▅▃▆▆█▄
val_loss,█▅▃▃▂▂▁▁▂▁
epoch,10
train_loss,0.64055
val_accuracy,0.57834
val_loss,0.63557


...

epoch,▁▂▃▃▄▅▆▆▇█
train_loss,█▃▂▂▁▁▁▁▁▁
val_accuracy,▆▅▄▁▄▂██▃█
val_loss,█▄▃▄▂▃▁█▂▁
epoch,10
train_loss,0.63457
val_accuracy,0.59996
val_loss,0.63038


wandb: Agent Starting Run: krsy3dsm with config:
wandb: 	activation: tanh
wandb: 	batch_size: 4096
wandb: 	hidden_size_1: 64
wandb: 	hidden_size_2: 32
wandb: 	hidden_size_3: 32
wandb: 	hidden_size_4: 64
wandb: 	learning_rate: 0.006054314872873776
wandb: 	n_layers: 4
wandb: 	optimizer: Adam


epoch,▁▂▃▃▄▅▆▆▇█
train_loss,█▂▂▂▁▁▁▁▁▁
val_accuracy,▅▅▃▅▃▁▃█▅▃
val_loss,█▃▃▂▂▂▁▄▂▁
epoch,10
train_loss,0.63557
val_accuracy,0.58602
val_loss,0.63094


wandb: Agent Starting Run: 6z83jcww with config:
wandb: 	activation: relu
wandb: 	batch_size: 4096
wandb: 	hidden_size_1: 64
wandb: 	hidden_size_2: 8
wandb: 	hidden_size_3: 8
wandb: 	hidden_size_4: 32
wandb: 	learning_rate: 0.006585751503467264
wandb: 	n_layers: 2
wandb: 	optimizer: SGD


epoch,▁▂▃▃▄▅▆▆▇█
train_loss,█▂▂▂▂▁▁▁▁▁
val_accuracy,▁▁▁▁▁▁▁▁▁▁
val_loss,█▇▆▅▄▃▂▂▁▁
epoch,10
train_loss,0.69187
val_accuracy,0.54022
val_loss,0.69167
